# gall_name 이외 건드리면 난리남
---
---
## !!! Selenium-Screenshot : 스크린샷 모듈 없으면 인스톨 필요함

#### [설명서]
-----
- gall_name : 갤러리명
- Kernel >> Restrat & Run ALL 누르면 끝

In [1]:
gall_name = "과자달리기_조현준" # <<<<<<<<<<<<<<<< 갤러리명 입력!!
                         # 1만개를 분할해서 돌릴때는 뒤에 숫자 1, 2, 3, ... 로 구분해줘야함

In [2]:
# # 스크린샷 모듈 설치 필요함 <<<<<<<<<<< 없으면 주석풀고 인스톨
# !pip install Selenium-Screenshot

In [3]:
import pandas as pd
read_df = pd.read_excel("./url_list/과자달리기_주소목록.xlsx") #<< 크롤링돌릴 엑셀 경로 입력
# read_df.head(5)

- 1만개씩이 너무 많은사람을 위한 기능( 바로 아래 셀 주석 풀기)

In [4]:
import pandas as pd
def df_split(dataframe,num) :  
    df_lst = []
    cnt = 0
    for i in range(num,len(dataframe),num) :
        #print(i)
        df_lst.append(dataframe.iloc[cnt:i,:])
        cnt += num

    df_lst.append(dataframe.iloc[cnt:, :])
    return df_lst

df_list = df_split(read_df ,50) # (분할할 데이터프레임 ,몇개씩 나눌지 입력)
len(df_list) # <<<<

4

- len(df_list)의 결과값을 확인하고
- 0부터 결과값-1 까지 숫자 바꿔가면서 돌리면됨

In [5]:
# # 기본 실행
# title_list = read_df.loc[:,"title"]
# url_list = read_df.loc[:,"url"]
# len(url_list), len(title_list)

# 1만개에서 분할을 했다?! 이걸 주석풀고 위에 주석처리
title_list = list(df_list[3].loc[:,"title"]) # 여기랑
url_list = list(df_list[3].loc[:,"url"]) # 여기 숫자들 같이 0부터 차례대로
len(url_list), len(title_list)

(12, 12)

In [6]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from Screenshot import Screenshot_Clipping
import requests
from bs4 import BeautifulSoup
import time
from os.path import getsize
import os.path
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from urllib.request import urlretrieve
from Screenshot import Screenshot_Clipping
from selenium.webdriver.chrome.options import Options
import datetime

### 1. 크롤링 기능 설계

In [7]:
# 현재날짜시간초로 고유값 라벨링 해주는 도구
def time_label():
    date_time = datetime.datetime.now()
    now_time = str(date_time).replace(" ","").replace("-","").replace(":","").replace(".","")
    return now_time

# 11칸 짜리 nan값 리스트 만들기
def nan_list_make():
    baseList = np.empty(12)
    baseList[:] = np.nan
    baseList = list(baseList)
    baseList
    return baseList

# 추가할 행 계산기
def row_count(img_list):
    row_cal = divmod(len(img_list),10)
    row_cnt = row_cal[0]
    if row_cal[1] > 0 :
        row_cnt += 1
    return row_cnt

# 리스트 10개씩 분할
def list_chunk(lst, n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

In [8]:
# 크롤링 코드
def image_download(BASE_URL, title):
    # 코드실행시각
    now_label = str(time_label())
    
    title_list = []
    title_url_list = []
    write_date_list = []
    img_path_list = []
    page_screenshot_list = []
    
    # 스샷찍을 도구----------------------
    options = wb.ChromeOptions() # 셀레니움 옵션
    options.add_argument("headless") # 브라우저 창 없이 실행
    options.add_argument("window-size=1920x1080")  # 브라우저 사이즈 조절
    #options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
    options.add_argument("disable-gpu")
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36") 
    options.add_argument("lang=ko_KR") 
    options.add_argument("Content-Type=application/json; charset=utf-8") 
    driver = wb.Chrome(options=options)
    #sshot=Screenshot_Clipping.Screenshot()

    # 헤더 설정 (필요한 대부분의 정보 제공 -> Bot Block 회피)
    headers = {
    "Connection" : "keep-alive",
    "Cache-Control" : "max-age=0",
    "sec-ch-ua-mobile" : "?0",
    "DNT" : "1",
    "Upgrade-Insecure-Requests" : "1",
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36",
    "Accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site" : "none",
    "Sec-Fetch-Mode" : "navigate",
    "Sec-Fetch-User" : "?1",
    "Sec-Fetch-Dest" : "document",
    "Accept-Encoding" : "gzip, deflate, br",
    "Accept-Language" : "ko-KR,ko;q=0.9"
    }

    # 페이지 가공
    res = requests.get(BASE_URL, headers=headers)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    
    write_date = soup.select_one("div.fl > span.gall_date").get("title") # 작성일 데이터 수집
    w_date = write_date.replace(" ", "").replace("-", "").replace(":", "") # 숫자제외 문자열 제거
    date_check = str(write_date).split(" ")[0].replace("-","")  # 날짜까지만
    
    
    # 페이지 스크린샷 -- 여기만 셀레니움으로 실행
    driver.get(BASE_URL)
    time.sleep(0.3)
    screenshot = driver.save_screenshot(f'./screenshot/sshot_{now_label}.jpg')
    time.sleep(0.3)
    driver.quit()
    
    # 데이터 수집,이미지 저장
    image_download_contents = soup.select("div.appending_file_box ul li")
    for li in image_download_contents:
        title_list.append(title)
        title_url_list.append(BASE_URL)
        write_date_list.append(write_date) # write_date : 게시글 작성 시간
        page_screenshot_list.append(f'sshot_{now_label}.jpg')

        img_tag = li.find('a', href=True)
        img_url = img_tag['href']

        file_ext = img_url.split('.')[-1] # ???

        #저장될 파일명
        savename = img_url.split("no=")[2].split(".")
        headers['Referer'] = BASE_URL
        response = requests.get(img_url, headers=headers)
        save_time = time_label()
        path = f"image/{savename[0]}_{save_time}.jpg"

        img_path_list.append(f"{savename[0]}_{save_time}.jpg")
        file = open(path , "wb")
        file.write(response.content)
        time.sleep(0.01)
        file.close()
        
    return title_list, title_url_list, write_date_list, img_path_list, page_screenshot_list

### 크롤링 실행 코드

In [9]:
# 게시글 크롤링
title_lst = [] # 게시글 제목 리스트
title_url_lst = [] # 게시글 주소
write_date_lst = [] # 게시글 작성일
img_path_lst = [] # 이미지 저장 이름
page_screenshot_lst = [] # 게시글 스크린샷
kind_lst = [] # 커뮤니티 종류
report_link_lst = [] # 신고 주소

error_url_lst = [] # 크롤링 에러 발생 페이지 주소
error_tit_lst = [] # 크롤링 에러 발생 페이지 제목

for i in tqdm_notebook(range(len(url_list))):
    link = url_list[i]
    title = title_list[i]
    
    try: # 이미지 다운로드하기
        titles, title_urls, write_dates, img_paths, page_screenshots = image_download(link, title)
        time.sleep(0.5)
        # 리턴값 리스트 추가
        title_lst += titles
        title_url_lst += title_urls
        write_date_lst += write_dates
        img_path_lst += img_paths
        page_screenshot_lst += page_screenshots
    except: #오류발생 페이지 목록 생성
        error_url_lst.append(link)
        error_tit_lst.append(title)

for i in range(len(title_lst)):
    kind_lst.append("D")
    report_link_lst.append("https://gall.dcinside.com/singo/?id=singo")

print("끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝")
print("끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝")
print("끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝")


<ipython-input-9-883c7de9dbf3>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(url_list))):


  0%|          | 0/12 [00:00<?, ?it/s]

C:\Users\guswn\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:1315: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  warnings.warn("name used for saved screenshot does not match file "


끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝
끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝
끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝


# 에러 관리

In [10]:
# 에러 발생한 페이지 목록 출력
print("에러 발생 "+str(len(error_url_lst))+"건")
for i in error_url_lst:
    print(i)

에러 발생 0건


In [11]:
# # 에러 많이 발생했을경우 이 셀을 주석 풀고 실행
# error_url_lst2 = []
# error_tit_lst2 = []

# title_lst = []
# title_url_lst = []
# write_date_lst = []
# img_path_lst = []
# page_screenshot_lst = []
# kind_lst = []
# report_link_lst = []

# for i in tqdm_notebook(range(len(error_url_lst))):
#     link = error_url_lst[i]
#     title = error_tit_lst[i]
    
#     try: # 이미지 다운로드하기
#         titles, title_urls, write_dates, img_paths, page_screenshots = image_download(link, title)
#         time.sleep(0.5)
#         # 리턴값 리스트 추가
#         title_lst += titles
#         title_url_lst += title_urls
#         write_date_lst += write_dates
#         img_path_lst += img_paths
#         page_screenshot_lst += page_screenshots
#     except: #오류발생 페이지 목록 생성
#         error_url_lst2.append(link)
#         error_tit_lst2.append(title)

# for i in range(len(title_lst)):
#     kind_lst.append("D")
#     report_link_lst.append("https://gall.dcinside.com/singo/?id=singo")

# print("끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝")
# print("끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝")
# print("끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝끝")
# print("에러 발생 "+str(len(error_url_lst))+"건")
# for i in error_url_lst2:
#     print(i)

## 크롤링 데이터 다루기

In [12]:
# 데이터 프레임화 시키기위한 덱셔너리 생성
dic = { # 크롤링 테이블
    'crawl_title' : title_lst,
    'crawl_url' : title_url_lst,
    'crawl_screenshot' : page_screenshot_lst,
    'crawl_report_link' : report_link_lst,
    'crawl_kind' : kind_lst
}

dic2 = {
    'crawl_title' : title_lst,
    'crawl_url' : title_url_lst,
    'img' : img_path_lst
}

# 크롤링 테이블용 DF
df = pd.DataFrame(dic)
df.drop_duplicates(subset=['crawl_url'], inplace=True) # 중복 컬럼 제거
df.reset_index(drop=True, inplace=True)

# 전처리용 DF
df2 = pd.DataFrame(dic2)

In [13]:
# df

In [14]:
# df2

In [15]:
# 이미지 테이블용 DF
add_title_list = df.loc[:,"crawl_title"]
add_url_list = df.loc[:,"crawl_url"]
nan_list = []

for t in range(len(add_title_list)) : 
    add_title = [add_title_list[t]] # []로 묶어서 리스트로 바꿔줌
    add_url = [add_url_list[t]]
    add_img_list = list(df2.loc[df.loc[t,"crawl_url"] == df2.loc[:,"crawl_url"], "img"])
    chunk_list = list_chunk(add_img_list, 10)

    for i in range(len(chunk_list)):
        lst = nan_list_make()
        add_lst = add_title + add_url + chunk_list[i]
        for j in range(len(add_lst)):
            lst[j] = add_lst[j]
        
        nan_list.append(lst)

df3 = pd.DataFrame(
    nan_list,
    columns=['crawl_title',"crawl_url",'img_file1','img_file2','img_file3',
             'img_file4','img_file5','img_file6','img_file7','img_file8',
             'img_file9','img_file10']
)

In [16]:
# df3

In [17]:
# df

In [18]:
# 엑셀 파일로 내보내기전에 인덱스 설정
df.set_index("crawl_title", inplace=True)
df3.set_index("crawl_title", inplace=True)

In [19]:
# 엑셀 내보내기
df.to_excel(f"./excel/{gall_name}_크롤링테이블.xlsx", encoding="utf-8")
df3.to_excel(f"./excel/{gall_name}_이미지테이블.xlsx", encoding="utf-8")